# HOMEWORK 1 - Fifteen Puzzle Game
**Student:** Alessandro Mattei

**Matricola:** Non Disponibile

**Email:** alessandro.mattei1@student.univaq.it

The main components used for the implementation will be presented:
   - Class Agent
   - Class Game (FifteenPuzzleGame)
   - Class GameRepresentation (FifteenPuzzleRepresentation)
   - Class State (StateFifteenPuzzleGame)
   - Heuristics (ManhattanDistance and MisplacedTittles)
   - Search Algorithm (A* and BestFirst) 

# Agent Class
Represents an agent that can act based on a given search algorithm and its current view of the world.

In [2]:
class Agent:
    """
    Represents an agent that can act based on a given search algorithm and its current view of the world.

    Attributes:
        search_algorithm: A search algorithm that the agent uses to make decisions.
        view: The agent's current view of the world.
        old_view: The agent's previous view of the world.
    """

    def __init__(self, search_algorithm, initial_state):
        """
        Initializes the Agent with a search algorithm and an initial state.

        :param search_algorithm: The search algorithm to be used by the agent.
        :param initial_state: The initial state of the world as perceived by the agent.
        """
        self.search_algorithm = search_algorithm
        self.view = initial_state
        self.old_view = None

    def do_action(self, current_state_world):
        """
        Updates the agent's view based on the current state of the world and the search algorithm.
        :param current_state_world: The current state of the world.
        :return: The updated view of the agent.
        """
        self.view = self.search_algorithm.search(current_state_world)
        self.old_view = current_state_world
        return self.view

# Game Representation
Represents the game board of game.
This class provides methods to check if the game board is solvable and to count the number of inversions
in the current state of the game board and moves that can be made.

In [3]:
import numpy as np


class FifteenPuzzleRepresentation:
    """
    Represents the game board of the Fifteen Puzzle game.

    This class provides methods to check if the game board is solvable and to count the number of inversions
    in the current state of the game board.

    Attributes:
        end_game_board (tuple): The final configuration of the game board.
        game_board (tuple): The current configuration of the game board.
    """

    def __init__(self, game_board=None):
        """
        Initializes the FifteenPuzzleRepresentation with a game board.
        :param game_board: The initial configuration of the game board.
                           Defaults to a pre-defined configuration.
        """
        self.end_game_board = tuple(range(1, 16)) + (0,)

        if game_board is None:
            self.game_board = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 15, 13, 14)
        else:
            self.game_board = game_board

    def game_is_solvable(self):
        """
        Determines if the current game board configuration is solvable.
        A game board is solvable if the number of inversions is even.
        :return: True if the game board is solvable, False otherwise.
        """
        # Counts the number of inversions in the state.
        inversions = self.count_inversions()
        # If the number of inversions is even, the state is solvable.
        return inversions % 2 == 0

    def count_inversions(self):
        """
        Counts the number of inversions in the current state of the game board.

        Inversions are pairs of tiles that are in the wrong order. For the game to be solvable,
        the number of inversions must be even.
        :return: The number of inversions in the game board.
        """
        state_array = np.array(self.game_board)
        state_array = state_array[state_array > 0]
        inversions = sum(np.sum(state_array[i + 1:] < state_array[i]) for i in range(len(state_array) - 1))
        return inversions

    def is_end_game(self):
        """
        Determines if the current game board configuration matches the end-game configuration.
        :return: True if the game board is in the end-game configuration, False otherwise.
        """
        return self.game_board == self.end_game_board

    def move_up(self):
        """
        Attempts to move the empty tile (0) up and returns a new game representation.

        If the move is not possible (i.e., the empty tile is already on the top row),
        the method returns None.
        :return: A new game representation if the move is possible; None otherwise.
        """
        board = np.array(self.game_board).reshape(4, 4)
        zero_row, zero_col = np.where(board == 0)  # Finds the position of the empty tile (0).
        new_row, new_col = zero_row + -1, zero_col
        if 0 <= new_row < 4 and 0 <= new_col < 4:
            new_board = board.copy()
            new_board[zero_row, zero_col], new_board[new_row, new_col] = new_board[new_row, new_col], new_board[
                zero_row, zero_col]
            return FifteenPuzzleRepresentation(game_board=tuple(new_board.flatten()))
        return None

    def move_down(self):
        """
        Attempts to move the empty tile (0) down and returns a new game representation.

        If the move is not possible (i.e., the empty tile is already on the bottom row),
        the method returns None.
        :return: A new game representation if the move is possible; None otherwise.
        """
        board = np.array(self.game_board).reshape(4, 4)
        zero_row, zero_col = np.where(board == 0)  # Finds the position of the empty tile (0).
        new_row, new_col = zero_row + 1, zero_col
        if 0 <= new_row < 4 and 0 <= new_col < 4:
            new_board = board.copy()
            new_board[zero_row, zero_col], new_board[new_row, new_col] = new_board[new_row, new_col], new_board[
                zero_row, zero_col]
            return FifteenPuzzleRepresentation(game_board=tuple(new_board.flatten()))
        return None

    def move_left(self):
        """
        Attempts to move the empty tile (0) to the left and returns a new game representation.

        If the move is not possible (i.e., the empty tile is already on the leftmost column),
        the method returns None.
        :return: A new game representation if the move is possible; None otherwise.
        """
        board = np.array(self.game_board).reshape(4, 4)
        zero_row, zero_col = np.where(board == 0)  # Finds the position of the empty tile (0).
        new_row, new_col = zero_row, zero_col + -1
        if 0 <= new_row < 4 and 0 <= new_col < 4:
            new_board = board.copy()
            new_board[zero_row, zero_col], new_board[new_row, new_col] = new_board[new_row, new_col], new_board[
                zero_row, zero_col]
            return FifteenPuzzleRepresentation(game_board=tuple(new_board.flatten()))
        return None

    def move_right(self):
        """
        Attempts to move the empty tile (0) to the right and returns a new game representation.

        If the move is not possible (i.e., the empty tile is already on the leftmost column),
        the method returns None.
        :return: A new game representation if the move is possible; None otherwise.
        """
        board = np.array(self.game_board).reshape(4, 4)
        zero_row, zero_col = np.where(board == 0)  # Finds the position of the empty tile (0).
        new_row, new_col = zero_row, zero_col + 1
        if 0 <= new_row < 4 and 0 <= new_col < 4:
            new_board = board.copy()
            new_board[zero_row, zero_col], new_board[new_row, new_col] = new_board[new_row, new_col], new_board[
                zero_row, zero_col]
            return FifteenPuzzleRepresentation(game_board=tuple(new_board.flatten()))
        return None

    def __eq__(self, other):
        if not isinstance(other, FifteenPuzzleRepresentation):
            return False
        return np.array_equal(self.game_board, other.game_board)

    def __ne__(self, other):
        return not self.__eq__(other)

    def __hash__(self):
        return hash(self.game_board)

    def __str__(self):
        return str(self.game_board)


# State
Represents a state in the Game.

In [4]:
class StateFifteenPuzzleGame:
    """
    Represents a state in the Fifteen Puzzle game.

    This class provides methods to determine if the state is an end-game configuration,
    print the game board, and compare states.

    Attributes:
        game_representation (FifteenPuzzleRepresentation): The current Representation of the game board.
        state_parent (StateFifteenPuzzleGame): The parent state from which this state was derived.
        move (str): The move that led to this state.
        h (float): The heuristic value of the state.
        g (float): The cost to reach the current state from the start state.
        f (float): The estimated cost of the cheapest solution through the current state.
    """

    def __init__(self, game_representation=None, state_parent=None, move=None):
        """
        Initializes the StateFifteenPuzzleGame with a game representation, parent state, and move.
        :param game_representation: The game board representation.
        :param state_parent: The parent state.
        :param move: The move that led to this state.
        """
        self.game_representation = game_representation
        self.state_parent = state_parent
        self.move = move
        self.h = None
        self.g = None
        self.f = None

        if self.game_representation is None:
            self.game_representation = FifteenPuzzleRepresentation()

    def is_end_game(self):
        """
        Determines if the current state matches the end-game configuration.
        :return: True if the state is an end-game configuration, False otherwise.
        """
        return self.game_representation.is_end_game()

    def print_board(self):
        """
        Returns a console representation of the game board of the current state.
        """
        for i in range(4):
            for j in range(4):
                print(self.game_representation.game_board[i * 4 + j], end="\t")
            print()
        print()

    def __eq__(self, other):
        if not isinstance(other, StateFifteenPuzzleGame):
            return False
        return self.game_representation.game_board == other.game_representation.game_board

    def __ne__(self, other):
        return not self.__eq__(other)

    def __hash__(self):
        return self.game_representation.__hash__()

    def __lt__(self, other):
        if self.f is None or other.f is None:
            raise ValueError("Cannot compare states")
        return self.f < other.f


# Game Class
This class provides methods to get neighbors of a given state

In [5]:
class FifteenPuzzleGame:
    """
    Represents the operations related to the Fifteen Puzzle game.
    This class provides methods to get neighbors of a given state, which are
    the possible states that can be reached by making valid moves from the current state.
    """

    def neighbors(self, state: StateFifteenPuzzleGame):
        """
        Computes the neighboring states that can be reached from the given state.
        The method calculates the possible states that can be reached by making
        the "UP", "DOWN", "LEFT", and "RIGHT" moves from the current state.
        :param state:  The current state of the Fifteen Puzzle game.
        :return: A set of neighboring states that can be reached from the given state.
        """
        neighbors_state = set()
        moves = {
            "UP": state.game_representation.move_up(),
            "DOWN": state.game_representation.move_down(),
            "LEFT": state.game_representation.move_left(),
            "RIGHT": state.game_representation.move_right(),
        }
        for move, new_state in moves.items():
            if new_state is not None:
                neighbors_state.add(
                    StateFifteenPuzzleGame(game_representation=new_state, state_parent=state, move=move))
        return neighbors_state


# Heuristics
## Manhattan Distance
Represents a heuristic for the Fifteen Puzzle game based on the Manhattan distance of tiles from their goal positions.

In [6]:
class ManhattanDistancePuzzleGame:
    """
    Represents a heuristic for the Fifteen Puzzle game based on the Manhattan distance of tiles from their goal positions.

    This heuristic computes the sum of the Manhattan distances of each tile from its goal position.
    The Manhattan distance for a tile is the sum of the absolute values of the horizontal and vertical distances
    between the tile's current position and its goal position.

    Attributes:
        goal_position (tuple): The goal configuration of the game board.
    """

    def __init__(self, goal_position):
        """
        Initializes the ManhattanDistancePuzzleGameOp heuristic with a goal position.
        :param goal_position: The goal configuration of the game board.
        """
        self.goal_position = goal_position

    def h(self, state: StateFifteenPuzzleGame):
        """
        Computes the heuristic value for a given state based on the Manhattan distance of tiles.

        The heuristic value is the sum of the Manhattan distances of each tile from its goal position.
        :param state: The current state of the Fifteen Puzzle game.
        :return: The heuristic value for the given state.
        """
        distance = 0  # Initialize the total distance to 0.

        for i in range(16):
            # Iterate through each tile on a 4x4 board (16 tiles in total).
            if state.game_representation.game_board[i] == 0:  # Ignore the empty tile.
                continue

            # Convert the current tile's index to its (x, y) position on the board.
            current_position = self._index_to_position(i)
            # Find the goal position of the current tile and convert its index to (x, y) position.
            goal_position = self._index_to_position(self.goal_position.index(state.game_representation.game_board[i]))

            # Calculate the Manhattan distance between the current position and the goal position.
            # The Manhattan distance is the sum of the absolute differences of their coordinates.
            distance += abs(current_position[0] - goal_position[0]) + abs(current_position[1] - goal_position[1])

        return distance  # Return the total Manhattan distance.

    def _index_to_position(self, index):
        return index // 4, index % 4  # Returns the row and column as a tuple.


## Misplaced Tittles
Represents a heuristic for the Fifteen Puzzle game based on the number of misplaced tiles.

In [7]:
class MisplacedTittlesPuzzleGame:
    """
    Represents a heuristic for the Fifteen Puzzle game based on the number of misplaced tiles.
    This heuristic computes the number of tiles that are not in their goal position, excluding the empty tile (0).

    Attributes:
        goal_position (tuple): The goal configuration of the game board.
    """

    def __init__(self, goal_position):
        """
        Initializes the MisplacedTilesPuzzleGame heuristic with a goal position.
        :param goal_position: (tuple) The goal configuration of the game board.
        """
        self.goal_position = goal_position

    def h(self, state: StateFifteenPuzzleGame):
        """
        Computes the heuristic value for a given state based on the number of misplaced tiles.

        The heuristic value is the number of tiles that are not in their goal position, excluding the empty tile (0).
        :param state: The current state of the Fifteen Puzzle game.
        :return: The heuristic value for the given state.
        """
        return sum(
            1 for s, g in zip(state.game_representation.game_board, self.goal_position)
            if s != g and s != 0
        )
    

# Search Algorithm
## A*

In [8]:
class AStar:
    """
    Represents the A* Search algorithm

    This class provides methods to perform the A* Search based on a given heuristic.
    The search selects states to expand based on a combined cost (f value) which is
    the sum of the actual cost to reach the state (g value) and the estimated cost
    from the state to the goal (h value).

    Attributes:
        heuristic: The heuristic used to estimate the cost from a given state to the goal state.
        game: The Fifteen Puzzle game.
        horizon (set): The set of states that are on the boundary of the explored space.
        explored (set): The set of states that have already been explored.
    """

    def __init__(self, game, heuristic):
        """
        Initializes the AStarOp search with a game and heuristic.
        :param game: The Fifteen Puzzle game.
        :param heuristic: The heuristic used to estimate the cost from a state to the goal state.
        """
        self.heuristic = heuristic
        self.game = game
        self.horizon = set()  # A set to store the frontier nodes yet to be explored.
        self.explored = set()  # A set to keep track of already explored nodes.

    def evaluate(self, state):
        """
        Computes the cost (g value), heuristic value (h value), and combined cost (f value) for a given state.

        The g value is the actual cost to reach the state, the h value is the estimated cost from the state to the goal,
        and the f value is the sum of the g and h values.
        :param state: The state for which the costs need to be computed.
        """
        if state.g is None:
            if state.state_parent is None:
                state.g = 0
            else:
                state.g = state.state_parent.g + 1  # fixed cost set to 1

            state.h = self.heuristic.h(state)

            state.f = state.g + state.h

    def pick(self):
        """
        Selects the state with the lowest combined cost (f value) from the horizon.
        :return: The state with the lowest combined cost (f value).
        """
        return min(self.horizon, key=lambda state: state.f)

    def search(self, state):
        """
        Expands the search from the given state by exploring its neighbors.

        The method evaluates neighboring states of the given state, calculates their costs,
        and adds them to the search horizon if they haven't been explored before. It also updates the
        set of explored states.
        :param state: The state from which the search should be expanded.
        """
        if state not in self.horizon:
            self.evaluate(state)
            self.horizon.add(state)

        self.explored.add(state)

        neighbors = self.game.neighbors(state)

        for neighbor in neighbors:
            if neighbor not in self.horizon and neighbor not in self.explored:
                self.evaluate(neighbor)
                if neighbor.is_end_game():
                    return neighbor
                self.horizon.add(neighbor)

        self.horizon.remove(state)

        if len(self.horizon) > 0:
            return self.pick()
        else:
            return None


## BestFist

In [9]:
class BestFirst:
    """
    Represents the Best-First Search algorithm

    This class provides methods to perform the Best-First Search based on a given heuristic.
    The search selects states to expand based on the lowest heuristic value.

    Attributes:
        heuristic: The heuristic used to estimate the cost from a given state to the goal state.
        game: The Fifteen Puzzle game.
        horizon (set): The set of states that are on the boundary of the explored space.
        explored (set): The set of states that have already been explored.
    """

    def __init__(self, game, heuristic):
        """
        Initializes the BestFirst search with a game and heuristic.
        :param game: The Fifteen Puzzle game.
        :param heuristic: The heuristic used to estimate the cost from a state to the goal state.
        """
        self.heuristic = heuristic
        self.game = game
        self.horizon = set()
        self.explored = set()

    def f_value(self, states):
        """
        Computes the heuristic value for a given set of states.
        :param states: A set of states for which the heuristic values need to be computed.
        """
        for state in states:
            if state.f is None:
                state.f = self.heuristic.h(state)

    def evaluate(self, state: StateFifteenPuzzleGame):
        """
        Computes the heuristic value for a specific state.
        :param state: The state for which the heuristic value needs to be computed.
        :return:
        """
        if state.f is None:
            state.f = self.heuristic.h(state)
            state.h = state.f

    def pick(self):
        """
        Selects the state with the lowest heuristic value from the horizon.
        :return: The state with the lowest heuristic value.
        """
        return min(self.horizon, key=lambda state: state.f)

    def search(self, state):
        """
        Expands the search from the given state, evaluating neighboring states and adding them to the horizon.
        :param state: The state from which the search should be expanded.
        :return: The state with the lowest heuristic value. None otherwise.
        """
        if state not in self.horizon:
            self.evaluate(state)
            self.horizon.add(state)

        self.explored.add(state)

        neighbors = self.game.neighbors(state)

        for neighbor in neighbors:
            if neighbor not in self.horizon and neighbor not in self.explored:
                self.evaluate(neighbor)
                if neighbor.is_end_game():
                    return neighbor
                self.horizon.add(neighbor)

        self.horizon.remove(state)

        if len(self.horizon) > 0:
            return self.pick()
        else:
            return None


# Method Main

In [10]:
import time

def backpath(node):
    states = [node]
    parent = node.state_parent
    while parent is not None:
        states += [parent]
        parent = parent.state_parent
    return reversed(states)


def main_fifteen_puzzle_game():
    state = StateFifteenPuzzleGame()
    heuristics = ManhattanDistancePuzzleGame(state.game_representation.end_game_board)
    heuristics1 = MisplacedTittlesPuzzleGame(state.game_representation.end_game_board)
    run_agent("BestFirst", "ManhattanDistance", BestFirst(FifteenPuzzleGame(), heuristics), StateFifteenPuzzleGame())
    run_agent("AStar", "ManhattanDistance", AStar(FifteenPuzzleGame(), heuristics), StateFifteenPuzzleGame())
    run_agent("BestFirst", "MisplacedTittles", BestFirst(FifteenPuzzleGame(), heuristics1), StateFifteenPuzzleGame())
    run_agent("AStar", "MisplacedTittles", AStar(FifteenPuzzleGame(), heuristics1), StateFifteenPuzzleGame())


def run_agent(search_algorithm_name, heuristics_name, search_algorithm, state):
    print(f"Run Agent with {search_algorithm_name} and {heuristics_name}")
    agent = Agent(search_algorithm, state)
    start_time = time.time()
    start_time_message = time.time()
    explored_states = 0
    if not state.game_representation.game_is_solvable():
        print("Not Solvable")
        return
    print("Solvable")

    while not state.is_end_game():
        state = agent.do_action(state)
        explored_states += 1
        if time.time() - start_time_message >= 1:
            print(f"Explored so far: {explored_states} in {time.time() - start_time:.0f}s")
            start_time_message = time.time()

        if state is None:
            print("The agent was unable to resolve the issue")
            return
    end_time = time.time()
    print(f"Result in : {(end_time - start_time) * 1000}ms")

    n = 0
    for s in backpath(state):
        s.print_board()
        n += 1
        if s.move is not None:
            print(f"The empty square was moved towards: {s.move}")
        print(f"h(n) = {s.h}")
        if search_algorithm_name == "AStar":
            print(f"g(n) = {s.g}")
            print(f"f(n) = {s.f}")
        print()
    print(f"Number of steps {n}")
    print(f"HORIZON: {len(search_algorithm.horizon)}")
    print(f"VISITED: {len(search_algorithm.explored)} \n")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")


if __name__ == '__main__':
    main_fifteen_puzzle_game()

Run Agent with BestFirst and ManhattanDistance
Solvable
Result in : 30.793190002441406ms
1	2	3	4	
5	6	7	8	
9	10	11	12	
0	15	13	14	

h(n) = 5

1	2	3	4	
5	6	7	8	
9	10	11	12	
15	0	13	14	

The empty square was moved towards: RIGHT
h(n) = 6

1	2	3	4	
5	6	7	8	
9	10	11	12	
15	13	0	14	

The empty square was moved towards: RIGHT
h(n) = 5

1	2	3	4	
5	6	7	8	
9	10	11	12	
15	13	14	0	

The empty square was moved towards: RIGHT
h(n) = 4

1	2	3	4	
5	6	7	8	
9	10	11	0	
15	13	14	12	

The empty square was moved towards: UP
h(n) = 5

1	2	3	4	
5	6	7	8	
9	10	0	11	
15	13	14	12	

The empty square was moved towards: LEFT
h(n) = 6

1	2	3	4	
5	6	7	8	
9	0	10	11	
15	13	14	12	

The empty square was moved towards: LEFT
h(n) = 7

1	2	3	4	
5	6	7	8	
9	13	10	11	
15	0	14	12	

The empty square was moved towards: DOWN
h(n) = 8

1	2	3	4	
5	6	7	8	
9	13	10	11	
0	15	14	12	

The empty square was moved towards: LEFT
h(n) = 7

1	2	3	4	
5	6	7	8	
0	13	10	11	
9	15	14	12	

The empty square was moved towards: UP
h(n) = 8

1	2	3	4	
5	6	